<a href="https://colab.research.google.com/github/luismiguelmartinluengo/Test_in_Colab/blob/main/Introduccion_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [91]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, col, count, mean, avg, sum, broadcast, desc, asc
from pyspark.sql import types

In [2]:
ruta_dataset_1 = 'drive/MyDrive/Colab Notebooks/Ficheros de prueba/csv1.txt'
ruta_dataset_2 = 'drive/MyDrive/Colab Notebooks/Ficheros de prueba/csv2.txt'

In [3]:
spark_session = (SparkSession.builder.appName("SparkSQLExampleApp").getOrCreate())

In [4]:
spark_session.version

'3.5.4'

In [5]:
import os
for item in os.listdir('drive/MyDrive/Colab Notebooks/Ficheros de prueba/'):
  print(item)

csv1.txt
csv2.txt


In [41]:
df1 = spark_session.read.csv(ruta_dataset_1, header = True, sep = ';')

In [7]:
df1.show()

+---+---+---+---+
|  a|  b|  c|  d|
+---+---+---+---+
|  1|  2|  3|  4|
|  5|  6|  7|  8|
|  9| 10| 11| 12|
| 13| 14| 15| 16|
+---+---+---+---+



In [8]:
df1.describe().show()

+-------+-----------------+-----------------+-----------------+-----------------+
|summary|                a|                b|                c|                d|
+-------+-----------------+-----------------+-----------------+-----------------+
|  count|                4|                4|                4|                4|
|   mean|              7.0|              8.0|              9.0|             10.0|
| stddev|5.163977794943222|5.163977794943222|5.163977794943222|5.163977794943222|
|    min|                1|               10|               11|               12|
|    max|                9|                6|                7|                8|
+-------+-----------------+-----------------+-----------------+-----------------+



In [9]:
df1.printSchema()

root
 |-- a: string (nullable = true)
 |-- b: string (nullable = true)
 |-- c: string (nullable = true)
 |-- d: string (nullable = true)



In [10]:
df1.isEmpty()

False

In [11]:
df1.columns

['a', 'b', 'c', 'd']

In [12]:
df1.count()

4

In [13]:
df1.head(2)

[Row(a='1', b='2', c='3', d='4'), Row(a='5', b='6', c='7', d='8')]

In [42]:
df2 = spark_session.read.csv(ruta_dataset_2, header = True, sep = ';')
df2.show(2)

+---+---+---+---+---+
|  a|  d|  e|  f|  g|
+---+---+---+---+---+
|  1| 17| 18| 19| 20|
|  5| 21| 22| 23| 24|
+---+---+---+---+---+
only showing top 2 rows



In [16]:
df1_2 = spark_session.read.csv(ruta_dataset_1, header = True, sep = ';')
dfea = df1.exceptAll(df1_2)
dfea.show()

+---+---+---+---+
|  a|  b|  c|  d|
+---+---+---+---+
+---+---+---+---+



In [17]:
dfea.describe().show()

+-------+----+----+----+----+
|summary|   a|   b|   c|   d|
+-------+----+----+----+----+
|  count|   0|   0|   0|   0|
|   mean|NULL|NULL|NULL|NULL|
| stddev|NULL|NULL|NULL|NULL|
|    min|NULL|NULL|NULL|NULL|
|    max|NULL|NULL|NULL|NULL|
+-------+----+----+----+----+



In [18]:
df1.select('a','c').limit(3).show()

+---+---+
|  a|  c|
+---+---+
|  1|  3|
|  5|  7|
|  9| 11|
+---+---+



In [19]:
df1.take(2)

[Row(a='1', b='2', c='3', d='4'), Row(a='5', b='6', c='7', d='8')]

In [20]:
df1.head(2)

[Row(a='1', b='2', c='3', d='4'), Row(a='5', b='6', c='7', d='8')]

In [21]:
filas = df1.limit(2).collect()
print(filas[0].asDict())
print(filas[1].asDict())

{'a': '1', 'b': '2', 'c': '3', 'd': '4'}
{'a': '5', 'b': '6', 'c': '7', 'd': '8'}


In [23]:
union = df1.union(df1_2)
union.show()

+---+---+---+---+
|  a|  b|  c|  d|
+---+---+---+---+
|  1|  2|  3|  4|
|  5|  6|  7|  8|
|  9| 10| 11| 12|
| 13| 14| 15| 16|
|  1|  2|  3|  4|
|  5|  6|  7|  8|
|  9| 10| 11| 12|
| 13| 14| 15| 16|
+---+---+---+---+



In [24]:
dfwc = df1.withColumns({'nueva_columna':when(df1.a == '1', 'cazado').otherwise('no cazado'),
                        'nueva_columna2':lit(6)})
dfwc.show()

+---+---+---+---+-------------+--------------+
|  a|  b|  c|  d|nueva_columna|nueva_columna2|
+---+---+---+---+-------------+--------------+
|  1|  2|  3|  4|       cazado|             6|
|  5|  6|  7|  8|    no cazado|             6|
|  9| 10| 11| 12|    no cazado|             6|
| 13| 14| 15| 16|    no cazado|             6|
+---+---+---+---+-------------+--------------+



In [25]:
dfwc.withColumnRenamed('nueva_columna', 'es_cazado').drop('nueva_columna2').show()

+---+---+---+---+---------+
|  a|  b|  c|  d|es_cazado|
+---+---+---+---+---------+
|  1|  2|  3|  4|   cazado|
|  5|  6|  7|  8|no cazado|
|  9| 10| 11| 12|no cazado|
| 13| 14| 15| 16|no cazado|
+---+---+---+---+---------+



In [26]:
union = union.dropDuplicates()#sin parámetros hace lo mismo que distinct
union = union.distinct()
union.show()

+---+---+---+---+
|  a|  b|  c|  d|
+---+---+---+---+
|  1|  2|  3|  4|
|  9| 10| 11| 12|
| 13| 14| 15| 16|
|  5|  6|  7|  8|
+---+---+---+---+



In [27]:
datos_nuevo_df = [('n11','n12','n13','n14'),
                  ('n21','n22','n23','n24')]
columnas_nuevo_df = union.columns
nuevo_df = spark_session.createDataFrame(data = datos_nuevo_df, schema = columnas_nuevo_df)
nuevo_df.show()

+---+---+---+---+
|  a|  b|  c|  d|
+---+---+---+---+
|n11|n12|n13|n14|
|n21|n22|n23|n24|
+---+---+---+---+



In [28]:
union = union.union(nuevo_df)
union.show()

+---+---+---+---+
|  a|  b|  c|  d|
+---+---+---+---+
|  1|  2|  3|  4|
|  9| 10| 11| 12|
| 13| 14| 15| 16|
|  5|  6|  7|  8|
|n11|n12|n13|n14|
|n21|n22|n23|n24|
+---+---+---+---+



In [29]:
union.createOrReplaceTempView('union')
df_select = spark_session.sql('select * from union where a like "%1%"')
df_select.show()

+---+---+---+---+
|  a|  b|  c|  d|
+---+---+---+---+
|  1|  2|  3|  4|
| 13| 14| 15| 16|
|n11|n12|n13|n14|
|n21|n22|n23|n24|
+---+---+---+---+



In [30]:
#Otra forma de filtro (como pandas)
df_filtrado = union.filter(col("a").contains("1"))
df_filtrado.show()

+---+---+---+---+
|  a|  b|  c|  d|
+---+---+---+---+
|  1|  2|  3|  4|
| 13| 14| 15| 16|
|n11|n12|n13|n14|
|n21|n22|n23|n24|
+---+---+---+---+



In [31]:
print(df_filtrado.collect())

[Row(a='1', b='2', c='3', d='4'), Row(a='13', b='14', c='15', d='16'), Row(a='n11', b='n12', c='n13', d='n14'), Row(a='n21', b='n22', c='n23', d='n24')]


In [32]:
df1.show()

+---+---+---+---+
|  a|  b|  c|  d|
+---+---+---+---+
|  1|  2|  3|  4|
|  5|  6|  7|  8|
|  9| 10| 11| 12|
| 13| 14| 15| 16|
+---+---+---+---+



In [33]:
dfnum = df1.withColumns({"a": df1.a.cast(types.IntegerType()),
                         "b": df1.b.cast(types.IntegerType()),
                         "d": df1.d.cast(types.FloatType())})
print(dfnum.dtypes)
dfnum.show()

[('a', 'int'), ('b', 'int'), ('c', 'string'), ('d', 'float')]
+---+---+---+----+
|  a|  b|  c|   d|
+---+---+---+----+
|  1|  2|  3| 4.0|
|  5|  6|  7| 8.0|
|  9| 10| 11|12.0|
| 13| 14| 15|16.0|
+---+---+---+----+



In [34]:
dfag = dfnum.groupBy('c').agg({'a':'sum', 'b':'mean', 'd':'max'})
dfag.show()

+---+------+------+------+
|  c|sum(a)|avg(b)|max(d)|
+---+------+------+------+
|  7|     5|   6.0|   8.0|
| 15|    13|  14.0|  16.0|
| 11|     9|  10.0|  12.0|
|  3|     1|   2.0|   4.0|
+---+------+------+------+



In [39]:
dfag2 = dfnum.groupBy('c').agg(sum("a").alias("suma_de_a"),
                               mean("b").alias("mediana de b"),
                               avg("d").alias("media de d"))
dfag2.show()

+---+---------+------------+----------+
|  c|suma_de_a|mediana de b|media de d|
+---+---------+------------+----------+
|  7|        5|         6.0|       8.0|
| 15|       13|        14.0|      16.0|
| 11|        9|        10.0|      12.0|
|  3|        1|         2.0|       4.0|
+---+---------+------------+----------+



In [43]:
join = df1.join(df2, on = 'a', how = 'inner')
join.show()

+---+---+---+---+---+---+---+---+
|  a|  b|  c|  d|  d|  e|  f|  g|
+---+---+---+---+---+---+---+---+
|  1|  2|  3|  4| 17| 18| 19| 20|
|  5|  6|  7|  8| 21| 22| 23| 24|
|  9| 10| 11| 12| 25| 26| 27| 28|
| 13| 14| 15| 16| 29| 30| 31| 32|
+---+---+---+---+---+---+---+---+



In [45]:
crossjoin = df1.crossJoin(df2.select('f','g'))
crossjoin.show()

+---+---+---+---+---+---+
|  a|  b|  c|  d|  f|  g|
+---+---+---+---+---+---+
|  1|  2|  3|  4| 19| 20|
|  1|  2|  3|  4| 23| 24|
|  1|  2|  3|  4| 27| 28|
|  1|  2|  3|  4| 31| 32|
|  5|  6|  7|  8| 19| 20|
|  5|  6|  7|  8| 23| 24|
|  5|  6|  7|  8| 27| 28|
|  5|  6|  7|  8| 31| 32|
|  9| 10| 11| 12| 19| 20|
|  9| 10| 11| 12| 23| 24|
|  9| 10| 11| 12| 27| 28|
|  9| 10| 11| 12| 31| 32|
| 13| 14| 15| 16| 19| 20|
| 13| 14| 15| 16| 23| 24|
| 13| 14| 15| 16| 27| 28|
| 13| 14| 15| 16| 31| 32|
+---+---+---+---+---+---+



In [50]:
f1_broadcasted = broadcast(df1)
#se hace broadcast con el dataframe más pequeño. Esto hace que el dataset se copie en todos los nodos
#y así la ejecución es más rápida.
f1_broadcasted.show()
join_with_broadcasted = f1_broadcasted.join(df2, on = 'a', how = 'inner')
join_with_broadcasted.show()

+---+---+---+---+
|  a|  b|  c|  d|
+---+---+---+---+
|  1|  2|  3|  4|
|  5|  6|  7|  8|
|  9| 10| 11| 12|
| 13| 14| 15| 16|
+---+---+---+---+

+---+---+---+---+---+---+---+---+
|  a|  b|  c|  d|  d|  e|  f|  g|
+---+---+---+---+---+---+---+---+
|  1|  2|  3|  4| 17| 18| 19| 20|
|  5|  6|  7|  8| 21| 22| 23| 24|
|  9| 10| 11| 12| 25| 26| 27| 28|
| 13| 14| 15| 16| 29| 30| 31| 32|
+---+---+---+---+---+---+---+---+



In [53]:
''' sirve para tener copia de un dataframe
además permite usar la notación de texto que pongo en la función select
'''
df1_alias = df1.alias('df_con_alias')
df1_alias.select('df_con_alias.a','df_con_alias.b').show()


+---+---+
|  a|  b|
+---+---+
|  1|  2|
|  5|  6|
|  9| 10|
| 13| 14|
+---+---+



In [62]:
#rollup aplica la función dentro de la jerarquía
df2_num = df2.alias('df2_num')
for columna in df2_num.columns:
  df2_num = df2_num.withColumn(columna, df2_num[columna].cast(types.IntegerType()))
#End for
rollup = df2_num.rollup('a','d').avg('f')
rollup.orderBy('a','d').show()

+----+----+------+
|   a|   d|avg(f)|
+----+----+------+
|NULL|NULL|  25.0|
|   1|NULL|  19.0|
|   1|  17|  19.0|
|   5|NULL|  23.0|
|   5|  21|  23.0|
|   9|NULL|  27.0|
|   9|  25|  27.0|
|  13|NULL|  31.0|
|  13|  29|  31.0|
+----+----+------+



In [64]:
#igual que rollup pero desarrolla toda la jerarquía hacia adelante y hacia atrás
cube = df2_num.cube('a','d').avg('f')
cube.orderBy('a','d').show()

+----+----+------+
|   a|   d|avg(f)|
+----+----+------+
|NULL|NULL|  25.0|
|NULL|  17|  19.0|
|NULL|  21|  23.0|
|NULL|  25|  27.0|
|NULL|  29|  31.0|
|   1|NULL|  19.0|
|   1|  17|  19.0|
|   5|NULL|  23.0|
|   5|  21|  23.0|
|   9|NULL|  27.0|
|   9|  25|  27.0|
|  13|NULL|  31.0|
|  13|  29|  31.0|
+----+----+------+



In [68]:
pivot_gb = df2_num.groupBy('a').pivot('d').avg('f')
pivot_gb.show()

+---+----+----+----+----+
|  a|  17|  21|  25|  29|
+---+----+----+----+----+
|  1|19.0|NULL|NULL|NULL|
| 13|NULL|NULL|NULL|31.0|
|  5|NULL|23.0|NULL|NULL|
|  9|NULL|NULL|27.0|NULL|
+---+----+----+----+----+



In [73]:
rollup_pivot = df2_num.rollup('a','d').avg('f').withColumnRenamed('avg(f)','f')
rollup_pivot = rollup_pivot.groupby('a').pivot('d').max('f')
rollup_pivot.show()

+----+----+----+----+----+----+
|   a|null|  17|  21|  25|  29|
+----+----+----+----+----+----+
|NULL|25.0|NULL|NULL|NULL|NULL|
|   1|19.0|19.0|NULL|NULL|NULL|
|  13|31.0|NULL|NULL|NULL|31.0|
|   5|23.0|NULL|23.0|NULL|NULL|
|   9|27.0|NULL|NULL|27.0|NULL|
+----+----+----+----+----+----+



In [74]:
cube_pivot = df2_num.cube('a','d').avg('f').withColumnRenamed('avg(f)','f')
cube_pivot = cube_pivot.groupby('a').pivot('d').max('f')
cube_pivot.show()

+----+----+----+----+----+----+
|   a|null|  17|  21|  25|  29|
+----+----+----+----+----+----+
|NULL|25.0|19.0|23.0|27.0|31.0|
|   1|19.0|19.0|NULL|NULL|NULL|
|  13|31.0|NULL|NULL|NULL|31.0|
|   5|23.0|NULL|23.0|NULL|NULL|
|   9|27.0|NULL|NULL|27.0|NULL|
+----+----+----+----+----+----+



In [93]:
sb = df1.select('a','b').union(df2.select('a','d').withColumnRenamed('d','b'))
sb.sort(['a','b'], ascending = [False, True]).show()
sb.orderBy(['a','b'], ascending = [False, True]).show()
sb.orderBy(asc('a'), desc('b')).show()


+---+---+
|  a|  b|
+---+---+
|  9| 10|
|  9| 25|
|  5| 21|
|  5|  6|
| 13| 14|
| 13| 29|
|  1| 17|
|  1|  2|
+---+---+

+---+---+
|  a|  b|
+---+---+
|  9| 10|
|  9| 25|
|  5| 21|
|  5|  6|
| 13| 14|
| 13| 29|
|  1| 17|
|  1|  2|
+---+---+

+---+---+
|  a|  b|
+---+---+
|  1|  2|
|  1| 17|
| 13| 29|
| 13| 14|
|  5|  6|
|  5| 21|
|  9| 25|
|  9| 10|
+---+---+



In [96]:
duplicate_dropped = sb.dropDuplicates(['a'])
duplicate_dropped.show()

+---+---+
|  a|  b|
+---+---+
|  1|  2|
| 13| 14|
|  5|  6|
|  9| 10|
+---+---+

